# GARCH & EGARCH Volatility Models: A Deep Dive for Quant Traders

## Learning Objectives

By the end of this notebook, you will master:

1. **Mathematical foundations** of ARCH, GARCH, and EGARCH models
2. **Geometric intuition** behind volatility clustering
3. **Step-by-step derivations** of key formulas
4. **Numerical examples** with hand calculations
5. **Trading applications** integrating RSI, divergences, W/M patterns
6. **Python implementation** from scratch and with libraries
7. **Risk management** strategies using volatility forecasts
8. **Common mistakes** and how to avoid them

## Prerequisites

- Basic time series concepts (AR, MA, ARMA)
- Maximum likelihood estimation
- Basic options pricing knowledge
- Python proficiency

## Notebook Structure

1. **Theory & Intuition** - Why volatility models matter
2. **ARCH Models** - The foundation
3. **GARCH Models** - Adding persistence
4. **EGARCH Models** - Asymmetric effects
5. **Trading Applications** - Real-world strategies
6. **Integration with Technical Analysis** - RSI, divergences, patterns
7. **Mini Project** - Complete volatility trading system
8. **Exercises** - Test your understanding

---

**Author**: Cristian Mendoza  
**Level**: Advanced  
**Estimated Time**: 4-6 hours

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Time series analysis
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import het_arch

# GARCH models
from arch import arch_model
from arch.univariate import GARCH, EGARCH, ConstantMean, ZeroMean

# Financial data
import yfinance as yf
from datetime import datetime, timedelta

# Technical analysis
try:
    import ta
    from ta.momentum import RSIIndicator
    from ta.trend import MACD
    from ta.volume import OnBalanceVolumeIndicator
except ImportError:
    print("Installing ta library...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'ta'])
    import ta
    from ta.momentum import RSIIndicator

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 11

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("✓ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

# Part 2: Theory & Intuition - Why Volatility Models Matter

## The Big Picture

**Key Question**: Why can't we just use standard deviation?

**Answer**: Financial returns exhibit **volatility clustering**:
- Calm periods follow calm periods
- Volatile periods follow volatile periods
- "Large changes tend to be followed by large changes, of either sign" - Benoit Mandelbrot

## Geometric Visualization

Think of volatility as the "size of waves" in the ocean:
```
Low Volatility Period:       High Volatility Period:
~~~~~~                       /\/\/\/\
      ~~~~~~                 \/\/\/\/
            ~~~~~~           /\/\/\/\

Calm, predictable           Wild, unpredictable
```

## Mathematical Motivation

**Problem with constant volatility**:

Traditional assumption: $r_t \sim N(\mu, \sigma^2)$ where $\sigma$ is constant.

**Reality**: $\sigma_t$ changes over time!

**Solution**: Model $\sigma_t^2$ as a time-varying function of past information.

## 💡 Core Insight

GARCH models say:
> "Tomorrow's volatility depends on:
> 1. Today's surprise (squared return)
> 2. Today's volatility
> 3. A long-run average volatility"

$$\sigma_t^2 = \omega + \alpha r_{t-1}^2 + \beta \sigma_{t-1}^2$$

Where:
- $\omega$ = baseline volatility
- $\alpha$ = news coefficient (how much new shocks matter)
- $\beta$ = persistence coefficient (how much history matters)

## Trading Implications

1. **Option Pricing**: Use $\sigma_t$ instead of historical $\sigma$
2. **Risk Management**: Adjust position sizes based on $\sigma_t$
3. **Mean Reversion**: High volatility eventually subsides
4. **Regime Detection**: Identify volatility regimes

# Part 1: Environment Setup

Import all necessary libraries for volatility modeling and trading analysis.

# Part 3: Visual Demonstration of Volatility Clustering

Let's load real data and see volatility clustering in action.

In [ ]:
# Download S&P 500 data
ticker = 'SPY'
start_date = '2020-01-01'
end_date = '2024-01-01'

print(f"Downloading {ticker} data from {start_date} to {end_date}...")
data = yf.download(ticker, start=start_date, end=end_date, progress=False)

# Calculate returns
data['Returns'] = data['Close'].pct_change() * 100  # In percentage
data = data.dropna()

print(f"\n✓ Data downloaded: {len(data)} trading days")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
print(f"\nReturns Statistics:")
print(f"  Mean: {data['Returns'].mean():.4f}%")
print(f"  Std Dev: {data['Returns'].std():.4f}%")
print(f"  Min: {data['Returns'].min():.4f}%")
print(f"  Max: {data['Returns'].max():.4f}%")
print(f"  Skewness: {data['Returns'].skew():.4f}")
print(f"  Kurtosis: {data['Returns'].kurtosis():.4f}")

In [ ]:
# Visualize volatility clustering
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# 1. Price chart
axes[0].plot(data.index, data['Close'], linewidth=1.5, color='steelblue')
axes[0].set_title('S&P 500 (SPY) Price Chart', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Price ($)')
axes[0].grid(alpha=0.3)

# 2. Returns
axes[1].plot(data.index, data['Returns'], linewidth=0.8, color='darkgreen', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].set_title('Daily Returns - Notice the Clustering!', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Returns (%)')
axes[1].grid(alpha=0.3)

# Highlight volatility clusters
axes[1].axvspan('2020-02-01', '2020-05-01', alpha=0.2, color='red', label='COVID-19 Volatility')
axes[1].legend()

# 3. Absolute returns (proxy for volatility)
axes[2].plot(data.index, np.abs(data['Returns']), linewidth=0.8, color='darkred', alpha=0.7)
axes[2].set_title('Absolute Returns (Volatility Proxy)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('|Returns| (%)')
axes[2].set_xlabel('Date')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nOBSERVATION:")
print("Notice how high volatility periods cluster together (especially Mar 2020).")
print("This is what GARCH models capture: volatility is NOT constant!")

# Part 4: ARCH Model - The Foundation

## Mathematical Definition

**ARCH(q)** (Autoregressive Conditional Heteroskedasticity):

$$r_t = \sigma_t \varepsilon_t$$

$$\sigma_t^2 = \omega + \sum_{i=1}^{q} \alpha_i r_{t-i}^2$$

Where:
- $r_t$ = return at time $t$
- $\varepsilon_t \sim N(0,1)$ = standardized innovation
- $\sigma_t^2$ = conditional variance (time-varying!)
- $\omega > 0$ = baseline variance
- $\alpha_i \geq 0$ = ARCH coefficients

## Key Properties

1. **Conditional variance** depends on past squared returns
2. **Unconditional variance** is constant (if stationary)
3. **Fat tails**: ARCH generates leptokurtic distributions
4. **Volatility clustering**: Large |returns| follow large |returns|

## ARCH(1) Example - Hand Calculation

Let's work through ARCH(1) step by step.

In [ ]:
# ARCH(1) Manual Calculation
print("="*60)
print("ARCH(1) MODEL: σ²_t = ω + α₁ × r²_{t-1}")
print("="*60)

# Parameters
omega = 0.01
alpha_1 = 0.3

# Initial conditions
r_0 = 0.02  # 2% initial return
sigma_0 = 0.02  # 2% initial volatility

# Random shocks
np.random.seed(42)
shocks = np.array([-0.5, 1.2, 0.3, -0.8, 0.5])

print(f"\nParameters: ω = {omega}, α₁ = {alpha_1}")
print(f"Initial: r₀ = {r_0:.4f} ({r_0*100:.2f}%), σ₀ = {sigma_0:.4f}\n")

# Simulate
volatilities = [sigma_0]
returns = [r_0]

for t, epsilon in enumerate(shocks, start=1):
    # Step 1: Calculate variance at time t
    r_prev = returns[-1]
    sigma_t_squared = omega + alpha_1 * (r_prev ** 2)
    sigma_t = np.sqrt(sigma_t_squared)
    
    # Step 2: Calculate return at time t
    r_t = sigma_t * epsilon
    
    volatilities.append(sigma_t)
    returns.append(r_t)
    
    # Display
    print(f"Period t={t}:")
    print(f"  σ²_{t} = {omega} + {alpha_1} × ({r_prev:.4f})²")
    print(f"       = {omega} + {alpha_1} × {r_prev**2:.6f}")
    print(f"       = {sigma_t_squared:.6f}")
    print(f"  σ_{t}  = √{sigma_t_squared:.6f} = {sigma_t:.4f} ({sigma_t*100:.2f}%)")
    print(f"  r_{t}  = {sigma_t:.4f} × {epsilon:.2f} = {r_t:.4f} ({r_t*100:.2f}%)")
    print()

# Summary
print("="*60)
print("SUMMARY - Volatility Over Time:")
print("="*60)
for t, (vol, ret) in enumerate(zip(volatilities, returns)):
    print(f"t={t}: σ={vol*100:6.2f}%  r={ret*100:7.2f}%")

# Find maximum volatility
max_vol_idx = np.argmax(volatilities)
print(f"\n MAXIMUM VOLATILITY at t={max_vol_idx}: {volatilities[max_vol_idx]*100:.2f}%")
print(f"\n KEY INSIGHT: Large returns increase future volatility!")
print(f"   This is 'volatility clustering' captured by ARCH.")

# Part 5: GARCH Model - Adding Persistence

## Motivation

**Problem with ARCH**: Need many lags (large q) to capture long-memory volatility

**Solution**: GARCH adds lagged conditional variance

## GARCH(p,q) Definition

$$\sigma_t^2 = \omega + \sum_{i=1}^{q} \alpha_i r_{t-i}^2 + \sum_{j=1}^{p} \beta_j \sigma_{t-j}^2$$

**Most Common: GARCH(1,1)**

$$\sigma_t^2 = \omega + \alpha r_{t-1}^2 + \beta \sigma_{t-1}^2$$

Where:
- $\omega$ = long-run variance component
- $\alpha$ = news coefficient (reaction to shocks)
- $\beta$ = persistence coefficient (memory of past volatility)

## Key Insight

GARCH(1,1) says tomorrow's volatility depends on:
1. **Long-run average** (ω)
2. **Yesterday's surprise** (α × r²)
3. **Yesterday's volatility** (β × σ²)

## Persistence Measure

$$\text{Persistence} = \alpha + \beta$$

- Close to 1 → Very persistent (slow decay)
- Far from 1 → Quick mean reversion

In [ ]:
# GARCH(1,1) Manual Calculation
print("="*70)
print("GARCH(1,1) MODEL: σ²_t = ω + α × r²_{t-1} + β × σ²_{t-1}")
print("="*70)

# Typical market parameters
omega = 0.00001
alpha = 0.08
beta = 0.90

print(f"\nParameters (typical for equity markets):")
print(f"  ω = {omega:.6f}")
print(f"  α = {alpha:.6f}")
print(f"  β = {beta:.6f}")
print(f"  Persistence (α + β) = {alpha + beta:.6f}")
print(f"  {'Very persistent!' if alpha + beta > 0.95 else 'Moderate persistence'}")

# Unconditional variance
var_uncond = omega / (1 - alpha - beta)
sigma_uncond = np.sqrt(var_uncond)
print(f"\nUnconditional variance: {var_uncond:.6f}")
print(f"Unconditional volatility: {sigma_uncond:.4f} ({sigma_uncond*100:.2f}%)")

# Initial conditions
sigma_0_sq = 0.0004  # Initial variance
r_0 = 0.03  # 3% return

# Shocks
shocks = np.array([1.5, 0.1, 0.1, -2.0, 0.2])

print(f"\nInitial: r₀ = {r_0:.4f}, σ₀² = {sigma_0_sq:.6f}\n")

# Simulate
sigma_sq = [sigma_0_sq]
returns = [r_0]

for t, eps in enumerate(shocks, start=1):
    r_prev = returns[-1]
    sigma_prev_sq = sigma_sq[-1]
    
    # GARCH equation
    sigma_t_sq = omega + alpha * r_prev**2 + beta * sigma_prev_sq
    sigma_t = np.sqrt(sigma_t_sq)
    r_t = sigma_t * eps
    
    sigma_sq.append(sigma_t_sq)
    returns.append(r_t)
    
    print(f"Period t={t}:")
    print(f"  σ²_{t} = {omega:.6f} + {alpha:.2f}×({r_prev:.4f})² + {beta:.2f}×{sigma_prev_sq:.6f}")
    print(f"       = {omega:.6f} + {alpha * r_prev**2:.6f} + {beta * sigma_prev_sq:.6f}")
    print(f"       = {sigma_t_sq:.6f}")
    print(f"  σ_{t}  = {sigma_t:.4f} ({sigma_t*100:.2f}%)")
    print(f"  r_{t}  = {sigma_t:.4f} × {eps:.2f} = {r_t:.4f} ({r_t*100:.2f}%)")
    print()

# Analysis
print("="*70)
print("COMPARISON: How volatility evolves")
print("="*70)
for t in range(len(sigma_sq)):
    sigma_pct = np.sqrt(sigma_sq[t]) * 100
    if t == 0:
        print(f"t={t}: σ={sigma_pct:6.2f}%  (initial)")
    else:
        print(f"t={t}: σ={sigma_pct:6.2f}%  r={returns[t]*100:7.2f}%  shock={shocks[t-1]:5.2f}")

print(f"\nKEY OBSERVATION:")
print(f"After large shock at t=1 (ε=1.5), volatility increased to {np.sqrt(sigma_sq[1])*100:.2f}%")
print(f"But it PERSISTS due to β term, decaying slowly back to long-run average")
print(f"This is why GARCH is better than ARCH for capturing persistent volatility")

# Part 6: Estimating GARCH with Real Data

## Maximum Likelihood Estimation (MLE)

The log-likelihood function for GARCH(1,1):

$$\mathcal{L}(\theta) = \sum_{t=1}^{T} \left[-\frac{1}{2}\log(2\pi) - \frac{1}{2}\log(\sigma_t^2) - \frac{r_t^2}{2\sigma_t^2}\right]$$

where $\theta = (\omega, \alpha, \beta)$

We maximize this to find optimal parameters.

In [ ]:
# Estimate GARCH(1,1) on S&P 500 returns
print("Estimating GARCH(1,1) model on S&P 500 returns...")
print("="*70)

# Use returns from earlier
returns_series = data['Returns'].dropna()

# Fit GARCH(1,1) model
model = arch_model(returns_series, vol='Garch', p=1, q=1, dist='normal')
result = model.fit(disp='off')

# Display results
print(result.summary())

# Extract parameters
params = result.params
omega_est = params['omega']
alpha_est = params['alpha[1]']
beta_est = params['beta[1]']

print("\n" + "="*70)
print("ESTIMATED PARAMETERS:")
print("="*70)
print(f"ω (omega)     = {omega_est:.8f}")
print(f"α (alpha)     = {alpha_est:.6f}")
print(f"β (beta)      = {beta_est:.6f}")
print(f"Persistence   = {alpha_est + beta_est:.6f}")

# Unconditional volatility
var_uncond_est = omega_est / (1 - alpha_est - beta_est)
sigma_uncond_est = np.sqrt(var_uncond_est)

print(f"\nUnconditional variance: {var_uncond_est:.6f}")
print(f"Unconditional vol (daily): {sigma_uncond_est:.4f} ({sigma_uncond_est*100:.2f}%)")
print(f"Annualized volatility: {sigma_uncond_est * np.sqrt(252):.4f} ({sigma_uncond_est * np.sqrt(252) * 100:.2f}%)")

print("\n" + "="*70)
print("INTERPRETATION:")
print("="*70)
print(f"1. High persistence ({alpha_est + beta_est:.4f}) means volatility is very sticky")
print(f"2. β > α means past volatility matters MORE than recent shocks")
print(f"3. Long-run daily vol ~{sigma_uncond_est*100:.2f}% or ~{sigma_uncond_est * np.sqrt(252) * 100:.1f}% annualized")

In [ ]:
# Extract and plot conditional volatility
conditional_vol = result.conditional_volatility

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot 1: Returns with conditional volatility
axes[0].plot(data.index, data['Returns'], alpha=0.5, linewidth=0.8, label='Returns')
axes[0].plot(data.index, conditional_vol, color='red', linewidth=2, label='Conditional Volatility (σ_t)')
axes[0].plot(data.index, -conditional_vol, color='red', linewidth=2)
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0].set_title('S&P 500 Returns with GARCH(1,1) Conditional Volatility', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Returns / Volatility (%)')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Just volatility
axes[1].plot(data.index, conditional_vol, color='darkred', linewidth=1.5)
axes[1].axhline(y=sigma_uncond_est, color='blue', linestyle='--', linewidth=2, 
                label=f'Long-run vol = {sigma_uncond_est*100:.2f}%')
axes[1].set_title('GARCH(1,1) Conditional Volatility Over Time', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Volatility (%)')
axes[1].set_xlabel('Date')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Highlight COVID-19 period
axes[1].axvspan('2020-02-01', '2020-05-01', alpha=0.2, color='red', label='COVID-19')

plt.tight_layout()
plt.show()

print("\nOBSERVATION:")
print("Notice how volatility spikes during market stress (COVID-19) and")
print("gradually reverts to the long-run average. This is GARCH in action!")

# Part 7: EGARCH Model - Capturing Asymmetry

## The Leverage Effect

**Observation**: Negative returns increase volatility MORE than positive returns of equal magnitude.

**Why?**
- Bad news creates panic
- Falling prices increase leverage ratios
- Margin calls amplify downward pressure

**Problem with GARCH**: It's symmetric! A +5% and -5% return have the same impact.

## EGARCH(1,1) Definition

$$\log(\sigma_t^2) = \omega + \alpha \left|z_{t-1}\right| + \gamma z_{t-1} + \beta \log(\sigma_{t-1}^2)$$

where $z_t = \frac{\varepsilon_t}{\sigma_t} = \frac{r_t}{\sigma_t}$ is the standardized residual.

**Key Parameter: γ (gamma)**
- γ < 0: Negative shocks increase volatility MORE (typical for stocks)
- γ = 0: Symmetric (like GARCH)
- γ > 0: Positive shocks increase volatility MORE (rare)

**Advantages of EGARCH**:
1. Captures leverage effect (asymmetry)
2. No need for parameter constraints (log ensures σ² > 0)
3. Better for stock markets

In [ ]:
# Estimate EGARCH(1,1)
print("Estimating EGARCH(1,1) model on S&P 500 returns...")
print("="*70)

model_egarch = arch_model(returns_series, vol='EGARCH', p=1, q=1, dist='normal')
result_egarch = model_egarch.fit(disp='off')

print(result_egarch.summary())

# Extract parameters
params_e = result_egarch.params
omega_e = params_e['omega']
alpha_e = params_e['alpha[1]']
gamma_e = params_e['gamma[1]']
beta_e = params_e['beta[1]']

print("\n" + "="*70)
print("EGARCH PARAMETERS:")
print("="*70)
print(f"ω (omega) = {omega_e:.6f}")
print(f"α (alpha) = {alpha_e:.6f}")
print(f"γ (gamma) = {gamma_e:.6f}  <- KEY: Asymmetry parameter")
print(f"β (beta)  = {beta_e:.6f}")

print("\n" + "="*70)
print("INTERPRETATION OF γ:")
print("="*70)
if gamma_e < 0:
    print(f"γ = {gamma_e:.6f} < 0")
    print("NEGATIVE shocks (market drops) increase volatility MORE than positive shocks")
    print("This confirms the LEVERAGE EFFECT in stock markets")
elif gamma_e > 0:
    print(f"γ = {gamma_e:.6f} > 0")
    print("POSITIVE shocks increase volatility more (unusual for stocks)")
else:
    print(f"γ ≈ 0: Symmetric response (like GARCH)")

# Compare GARCH vs EGARCH
print("\n" + "="*70)
print("GARCH vs EGARCH COMPARISON:")
print("="*70)
print(f"{'Model':<15} {'ω':<12} {'α':<10} {'β':<10} {'γ':<10} {'Log-Lik':<12}")
print("-"*70)
print(f"{'GARCH(1,1)':<15} {omega_est:<12.8f} {alpha_est:<10.6f} {beta_est:<10.6f} {'N/A':<10} {result.loglikelihood:<12.2f}")
print(f"{'EGARCH(1,1)':<15} {omega_e:<12.8f} {alpha_e:<10.6f} {beta_e:<10.6f} {gamma_e:<10.6f} {result_egarch.loglikelihood:<12.2f}")

if result_egarch.loglikelihood > result.loglikelihood:
    print(f"\nEGARCH has HIGHER log-likelihood → Better fit for this data")
else:
    print(f"\nGARCH has HIGHER log-likelihood → Better fit for this data")

In [ ]:
# Demonstrate asymmetry with simulation
print("="*70)
print("DEMONSTRATING ASYMMETRY: +5% vs -5% shock")
print("="*70)

# Initial conditions
sigma_init = 0.02  # 2% daily vol

def simulate_egarch_shock(shock, sigma_init, omega, alpha, gamma, beta, periods=10):
    """Simulate EGARCH response to a shock"""
    log_sigma2 = np.zeros(periods + 1)
    log_sigma2[0] = np.log(sigma_init**2)
    
    for t in range(1, periods + 1):
        if t == 1:
            z = shock / sigma_init
        else:
            z = 0  # No more shocks
        
        log_sigma2[t] = omega + alpha * abs(z) + gamma * z + beta * log_sigma2[t-1]
    
    return np.exp(log_sigma2 / 2)

# Positive shock
shock_pos = 0.05  # +5%
sigma_pos = simulate_egarch_shock(shock_pos, sigma_init, omega_e, alpha_e, gamma_e, beta_e)

# Negative shock
shock_neg = -0.05  # -5%
sigma_neg = simulate_egarch_shock(shock_neg, sigma_init, omega_e, alpha_e, gamma_e, beta_e)

# Display results
print(f"\nInitial volatility: {sigma_init*100:.2f}%\n")
print(f"After +5% shock:")
print(f"  Day 1: σ = {sigma_pos[1]*100:.2f}% (increase of {(sigma_pos[1]/sigma_init - 1)*100:.1f}%)")

print(f"\nAfter -5% shock:")
print(f"  Day 1: σ = {sigma_neg[1]*100:.2f}% (increase of {(sigma_neg[1]/sigma_init - 1)*100:.1f}%)")

diff = sigma_neg[1] - sigma_pos[1]
print(f"\nDIFFERENCE: {diff*100:.3f}%")
print(f"The NEGATIVE shock increased volatility by {diff/sigma_pos[1]*100:.1f}% MORE")
print(f"This is the leverage effect captured by γ = {gamma_e:.4f}")

# Plot comparison
fig, ax = plt.subplots(figsize=(12, 6))
periods = range(len(sigma_pos))
ax.plot(periods, sigma_pos * 100, 'g-', marker='o', linewidth=2, markersize=6, label='+5% shock')
ax.plot(periods, sigma_neg * 100, 'r-', marker='o', linewidth=2, markersize=6, label='-5% shock')
ax.axhline(y=sigma_init*100, color='blue', linestyle='--', alpha=0.5, label='Initial vol')
ax.set_xlabel('Days after shock')
ax.set_ylabel('Volatility (%)')
ax.set_title('EGARCH Asymmetric Response: Negative Shocks Hit Harder', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nKEY TAKEAWAY:")
print("EGARCH captures the asymmetry in real markets where crashes increase")
print("volatility more than rallies. This is crucial for risk management!")

---

## Part 8: Volatility Forecasting

Now we generate multi-step ahead volatility forecasts, which are critical for:

- **Position sizing**: Scale positions inversely with forecasted volatility
- **Risk budgeting**: Allocate risk across assets based on vol forecasts
- **Option pricing**: Implied vol comparison with model forecasts
- **Dynamic hedging**: Adjust hedge ratios as volatility changes

GARCH models provide analytical forecasts for conditional variance:

**GARCH(1,1) h-step ahead forecast:**

$$\hat{\sigma}^2_{t+h|t} = \sigma^2_{LR} + (\alpha + \beta)^h \left( \hat{\sigma}^2_{t+1|t} - \sigma^2_{LR} \right)$$

Where:
- $\sigma^2_{LR} = \frac{\omega}{1 - \alpha - \beta}$ is the long-run variance
- $(\alpha + \beta)^h$ controls mean reversion speed
- As $h \to \infty$, forecast converges to $\sigma^2_{LR}$

**Key properties:**
- Forecasts decay exponentially to long-run mean
- High persistence ($\alpha + \beta \approx 1$) means slow decay
- Recent shocks have long-lasting impact on forecasts

In [ ]:
# Generate multi-step ahead forecasts
horizon = 20  # Forecast 20 days ahead

# GARCH forecasts
forecasts_garch = result_garch.forecast(horizon=horizon, start=0)
variance_forecast_garch = forecasts_garch.variance.iloc[-1]  # Last available forecast

# EGARCH forecasts
forecasts_egarch = result_egarch.forecast(horizon=horizon, start=0)
variance_forecast_egarch = forecasts_egarch.variance.iloc[-1]

# Convert to volatility (standard deviation)
vol_forecast_garch = np.sqrt(variance_forecast_garch)
vol_forecast_egarch = np.sqrt(variance_forecast_egarch)

print("="*70)
print(f"VOLATILITY FORECASTS (Next {horizon} Days)")
print("="*70)
print(f"\nLast observation date: {returns_series.index[-1].strftime('%Y-%m-%d')}")
print(f"Last realized volatility: {conditional_vol_garch.iloc[-1]*100:.2f}%\n")

print("GARCH(1,1) Forecasts:")
print("-" * 40)
for h in [1, 5, 10, 20]:
    print(f"  t+{h:2d}: {vol_forecast_garch.iloc[h-1]*100:5.2f}%")

print("\nEGARCH(1,1) Forecasts:")
print("-" * 40)
for h in [1, 5, 10, 20]:
    print(f"  t+{h:2d}: {vol_forecast_egarch.iloc[h-1]*100:5.2f}%")

# Calculate long-run volatility
lr_vol_garch = np.sqrt(omega_g / (1 - alpha_g - beta_g))
print(f"\nLong-run volatility (GARCH): {lr_vol_garch*100:.2f}%")

# Show mean reversion
persistence = alpha_g + beta_g
print(f"\nPersistence (α + β): {persistence:.4f}")
print(f"Half-life of shocks: {np.log(0.5) / np.log(persistence):.1f} days")
print("(Time for a shock to decay to 50% of its initial impact)")

# Visualize forecasts
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Forecast paths
ax1.plot(range(1, horizon+1), vol_forecast_garch * 100, 
         'b-', marker='o', linewidth=2, markersize=5, label='GARCH(1,1)')
ax1.plot(range(1, horizon+1), vol_forecast_egarch * 100, 
         'r-', marker='s', linewidth=2, markersize=5, label='EGARCH(1,1)')
ax1.axhline(y=lr_vol_garch*100, color='blue', linestyle='--', 
            alpha=0.5, label='Long-run vol (GARCH)')
ax1.axhline(y=conditional_vol_garch.iloc[-1]*100, color='green', 
            linestyle=':', alpha=0.7, label='Current realized vol')
ax1.set_xlabel('Forecast Horizon (days)')
ax1.set_ylabel('Volatility (%)')
ax1.set_title('Multi-Step Volatility Forecasts', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot 2: Forecast decay (relative to long-run)
decay_garch = (vol_forecast_garch - lr_vol_garch) / lr_vol_garch * 100
decay_egarch = (vol_forecast_egarch - lr_vol_garch) / lr_vol_garch * 100

ax2.plot(range(1, horizon+1), decay_garch, 
         'b-', marker='o', linewidth=2, markersize=5, label='GARCH(1,1)')
ax2.plot(range(1, horizon+1), decay_egarch, 
         'r-', marker='s', linewidth=2, markersize=5, label='EGARCH(1,1)')
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax2.set_xlabel('Forecast Horizon (days)')
ax2.set_ylabel('Deviation from Long-Run Vol (%)')
ax2.set_title('Mean Reversion: Forecast Decay to Long-Run Level', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nINTERPRETATION:")
print("Both models show mean reversion, but EGARCH forecasts differ slightly")
print("due to asymmetry. In trading, we use these forecasts to:")
print("  1. Scale position sizes (higher vol → smaller positions)")
print("  2. Adjust stop losses (wider stops in high vol regimes)")
print("  3. Compare with implied vol for options trading opportunities")

---

## Part 9: Trading Application - Volatility-Based Position Sizing

Now we implement a practical trading strategy that uses GARCH forecasts to dynamically adjust position sizes. This is a critical risk management technique used by professional traders.

**Strategy Logic:**

1. **Fixed Risk per Trade**: Target 2% portfolio risk per position
2. **Volatility-Adjusted Sizing**: 
   $$\text{Position Size} = \frac{\text{Risk Capital}}{\text{Forecasted Volatility} \times \text{Price}}$$
3. **Stop-Loss Placement**: Set stops at $2 \times \sigma_{forecast}$
4. **Capital Preservation**: Smaller positions in high-vol regimes

**Example:**
- Portfolio: $100,000
- Risk per trade: 2% = $2,000
- Forecasted vol: 1.5% daily
- Price: $400
- Position size: $2,000 / (0.015 × $400) = 333 shares
- Stop distance: $400 × 2 × 0.015 = $12 per share

This approach ensures consistent risk exposure across different market regimes.

In [ ]:
# Implement volatility-based position sizing
portfolio_value = 100000  # $100k portfolio
risk_per_trade = 0.02     # 2% risk per trade
risk_capital = portfolio_value * risk_per_trade  # $2,000

# Get price and forecasted volatility
prices = data['Close'].values
dates = data.index

# Use one-day ahead volatility forecasts
vol_forecast_1d_garch = np.sqrt(forecasts_garch.variance.iloc[:, 0])

# Calculate position sizes
position_sizes = []
stop_distances = []
stop_multiplier = 2.0  # Stop at 2 standard deviations

for i in range(len(prices) - 20, len(prices)):  # Last 20 days
    price = prices[i]
    vol = vol_forecast_1d_garch.iloc[i]
    
    # Position size calculation
    dollar_risk_per_share = stop_multiplier * vol * price
    shares = int(risk_capital / dollar_risk_per_share)
    position_value = shares * price
    
    # Stop loss distance
    stop_distance = stop_multiplier * vol * price
    
    position_sizes.append({
        'Date': dates[i],
        'Price': price,
        'Volatility': vol,
        'Shares': shares,
        'Position_Value': position_value,
        'Stop_Distance': stop_distance,
        'Stop_Loss_Price': price - stop_distance
    })

# Convert to DataFrame
df_positions = pd.DataFrame(position_sizes)

print("="*80)
print("VOLATILITY-ADJUSTED POSITION SIZING")
print("="*80)
print(f"\nPortfolio Value: ${portfolio_value:,.0f}")
print(f"Risk per Trade: {risk_per_trade*100:.1f}% = ${risk_capital:,.0f}")
print(f"Stop Distance: {stop_multiplier}x forecasted volatility\n")

print("Last 10 Trading Days:")
print("-" * 80)
print(df_positions.tail(10).to_string(index=False))

# Calculate summary statistics
print("\n" + "="*80)
print("POSITION SIZING STATISTICS")
print("="*80)
print(f"\nAverage Position Size: ${df_positions['Position_Value'].mean():,.0f} "
      f"({df_positions['Position_Value'].mean()/portfolio_value*100:.1f}% of portfolio)")
print(f"Min Position: ${df_positions['Position_Value'].min():,.0f} "
      f"({df_positions['Position_Value'].min()/portfolio_value*100:.1f}% of portfolio)")
print(f"Max Position: ${df_positions['Position_Value'].max():,.0f} "
      f"({df_positions['Position_Value'].max()/portfolio_value*100:.1f}% of portfolio)")

print(f"\nAverage Stop Distance: ${df_positions['Stop_Distance'].mean():.2f} "
      f"({df_positions['Stop_Distance'].mean()/df_positions['Price'].mean()*100:.2f}%)")

# Visualize position sizing over time
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Volatility over time
ax1 = axes[0]
ax1.plot(df_positions['Date'], df_positions['Volatility'] * 100, 
         'r-', linewidth=2, marker='o', markersize=4)
ax1.set_ylabel('Forecasted Vol (%)', color='r')
ax1.tick_params(axis='y', labelcolor='r')
ax1.set_title('Dynamic Position Sizing Based on GARCH Forecasts', 
              fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

# Plot 2: Position size (number of shares)
ax2 = axes[1]
ax2.bar(df_positions['Date'], df_positions['Shares'], color='blue', alpha=0.7)
ax2.set_ylabel('Position Size (shares)')
ax2.set_title('Shares Allocated (Inverse Relationship with Volatility)', 
              fontsize=12, fontweight='bold')
ax2.grid(alpha=0.3, axis='y')

# Plot 3: Position value vs. stop distance
ax3 = axes[2]
ax3_twin = ax3.twinx()
ax3.bar(df_positions['Date'], df_positions['Position_Value'], 
        color='green', alpha=0.5, label='Position Value')
ax3_twin.plot(df_positions['Date'], df_positions['Stop_Distance'], 
              'r-', marker='o', linewidth=2, markersize=4, label='Stop Distance')
ax3.set_ylabel('Position Value ($)', color='green')
ax3_twin.set_ylabel('Stop Distance ($)', color='red')
ax3.tick_params(axis='y', labelcolor='green')
ax3_twin.tick_params(axis='y', labelcolor='red')
ax3.set_title('Position Value and Stop Loss Distance', fontsize=12, fontweight='bold')
ax3.set_xlabel('Date')

plt.tight_layout()
plt.show()

print("\nKEY INSIGHTS:")
print("1. HIGH VOLATILITY → Smaller positions (fewer shares)")
print("2. LOW VOLATILITY → Larger positions (more shares)")
print("3. This maintains CONSTANT RISK of $2,000 per trade")
print("4. Stop losses automatically widen/narrow with volatility")
print("5. Professional risk management: 'Size to survive, trade to thrive'")

---

## Part 10: Integration with Technical Indicators - RSI + GARCH

We now combine **GARCH volatility regimes** with **RSI signals** to create a sophisticated trading system that:

1. Identifies overbought/oversold conditions (RSI)
2. Filters trades by volatility regime (GARCH)
3. Sizes positions dynamically (volatility-adjusted)

**Trading Rules:**

**Entry Signals:**
- **Long**: RSI < 30 (oversold) AND volatility is NOT extreme (σ < 3%)
- **Short**: RSI > 70 (overbought) AND volatility is NOT extreme

**Risk Management:**
- Position sizing: Inversely proportional to forecasted volatility
- Stop-loss: 2 × σ from entry price
- Exit: RSI crosses 50 (neutral) or stop hit

**Rationale:**
- Extreme volatility often precedes trend continuation (not reversal)
- Mean reversion works best in normal volatility regimes
- GARCH forecasts help avoid false signals during market stress

In [ ]:
# Calculate RSI
def calculate_rsi(prices, period=14):
    """Calculate RSI indicator"""
    deltas = np.diff(prices)
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)
    
    # Initial average
    avg_gain = np.mean(gains[:period])
    avg_loss = np.mean(losses[:period])
    
    rsi = np.zeros(len(prices))
    rsi[:period] = np.nan
    
    # Calculate RSI
    for i in range(period, len(prices)):
        avg_gain = (avg_gain * (period - 1) + gains[i - 1]) / period
        avg_loss = (avg_loss * (period - 1) + losses[i - 1]) / period
        
        if avg_loss == 0:
            rsi[i] = 100
        else:
            rs = avg_gain / avg_loss
            rsi[i] = 100 - (100 / (1 + rs))
    
    return rsi

# Calculate RSI on close prices
rsi = calculate_rsi(prices)

# Create trading signals
df_trading = pd.DataFrame({
    'Date': dates,
    'Price': prices,
    'RSI': rsi,
    'Volatility': np.concatenate([np.full(len(prices) - len(conditional_vol_garch), np.nan), 
                                   conditional_vol_garch.values])
})

# Drop NaN rows
df_trading = df_trading.dropna()

# Generate signals
vol_threshold = 0.03  # 3% daily vol threshold

df_trading['Signal'] = 0
df_trading.loc[(df_trading['RSI'] < 30) & 
               (df_trading['Volatility'] < vol_threshold), 'Signal'] = 1  # Long
df_trading.loc[(df_trading['RSI'] > 70) & 
               (df_trading['Volatility'] < vol_threshold), 'Signal'] = -1  # Short

# Calculate returns from signals
df_trading['Next_Return'] = df_trading['Price'].pct_change().shift(-1)
df_trading['Strategy_Return'] = df_trading['Signal'] * df_trading['Next_Return']

# Calculate cumulative returns
df_trading['Cumulative_Market'] = (1 + df_trading['Next_Return']).cumprod()
df_trading['Cumulative_Strategy'] = (1 + df_trading['Strategy_Return']).cumprod()

# Performance metrics
total_signals = (df_trading['Signal'] != 0).sum()
long_signals = (df_trading['Signal'] == 1).sum()
short_signals = (df_trading['Signal'] == -1).sum()

strategy_return = df_trading['Cumulative_Strategy'].iloc[-1] - 1
market_return = df_trading['Cumulative_Market'].iloc[-1] - 1
sharpe_ratio = df_trading['Strategy_Return'].mean() / df_trading['Strategy_Return'].std() * np.sqrt(252)

print("="*70)
print("RSI + GARCH TRADING STRATEGY RESULTS")
print("="*70)
print(f"\nStrategy: Long when RSI<30, Short when RSI>70")
print(f"Volatility Filter: Only trade when σ < {vol_threshold*100:.1f}%")
print(f"\nPeriod: {df_trading['Date'].iloc[0].strftime('%Y-%m-%d')} to "
      f"{df_trading['Date'].iloc[-1].strftime('%Y-%m-%d')}")
print(f"Total Trading Days: {len(df_trading)}")

print(f"\nSIGNAL SUMMARY:")
print(f"  Total signals: {total_signals}")
print(f"  Long signals: {long_signals}")
print(f"  Short signals: {short_signals}")
print(f"  Days in market: {total_signals / len(df_trading) * 100:.1f}%")

print(f"\nPERFORMANCE:")
print(f"  Buy & Hold Return: {market_return*100:+.2f}%")
print(f"  Strategy Return: {strategy_return*100:+.2f}%")
print(f"  Outperformance: {(strategy_return - market_return)*100:+.2f}%")
print(f"  Sharpe Ratio: {sharpe_ratio:.2f}")

# Visualize trading signals and performance
fig, axes = plt.subplots(4, 1, figsize=(14, 14))

# Plot 1: Price with signals
ax1 = axes[0]
ax1.plot(df_trading['Date'], df_trading['Price'], 'k-', linewidth=1.5, label='SPY Price')
long_dates = df_trading[df_trading['Signal'] == 1]['Date']
long_prices = df_trading[df_trading['Signal'] == 1]['Price']
short_dates = df_trading[df_trading['Signal'] == -1]['Date']
short_prices = df_trading[df_trading['Signal'] == -1]['Price']
ax1.scatter(long_dates, long_prices, color='green', marker='^', s=100, 
            label='Long Signal', zorder=5)
ax1.scatter(short_dates, short_prices, color='red', marker='v', s=100, 
            label='Short Signal', zorder=5)
ax1.set_ylabel('Price ($)')
ax1.set_title('Trading Signals: RSI + GARCH Volatility Filter', 
              fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot 2: RSI with levels
ax2 = axes[1]
ax2.plot(df_trading['Date'], df_trading['RSI'], 'purple', linewidth=1.5)
ax2.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
ax2.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
ax2.axhline(y=50, color='gray', linestyle='-', alpha=0.5)
ax2.fill_between(df_trading['Date'], 70, 100, alpha=0.1, color='red')
ax2.fill_between(df_trading['Date'], 0, 30, alpha=0.1, color='green')
ax2.set_ylabel('RSI')
ax2.set_title('RSI Oscillator', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)
ax2.set_ylim(0, 100)

# Plot 3: Volatility
ax3 = axes[2]
ax3.plot(df_trading['Date'], df_trading['Volatility'] * 100, 'orange', linewidth=1.5)
ax3.axhline(y=vol_threshold*100, color='red', linestyle='--', linewidth=2, 
            label=f'Vol Threshold ({vol_threshold*100:.0f}%)')
ax3.fill_between(df_trading['Date'], vol_threshold*100, 
                 df_trading['Volatility'].max()*100, alpha=0.2, color='red')
ax3.set_ylabel('Volatility (%)')
ax3.set_title('GARCH Conditional Volatility (Regime Filter)', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

# Plot 4: Cumulative returns
ax4 = axes[3]
ax4.plot(df_trading['Date'], (df_trading['Cumulative_Market'] - 1) * 100, 
         'gray', linewidth=2, label='Buy & Hold', alpha=0.7)
ax4.plot(df_trading['Date'], (df_trading['Cumulative_Strategy'] - 1) * 100, 
         'blue', linewidth=2, label='RSI + GARCH Strategy')
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax4.set_ylabel('Cumulative Return (%)')
ax4.set_xlabel('Date')
ax4.set_title('Strategy vs Buy & Hold Performance', fontsize=12, fontweight='bold')
ax4.legend()
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKEY TAKEAWAYS:")
print("1. Volatility filter reduces false signals during market stress")
print("2. RSI mean reversion works better in stable volatility regimes")
print("3. Combined approach improves risk-adjusted returns")
print("4. Integration of technical + statistical models = superior edge")

---

## Part 11: Advanced Integration - Divergence Analysis with GARCH

**Divergence** occurs when price and an indicator (like RSI) move in opposite directions, signaling potential reversals.

**Types of Divergences:**

1. **Regular Bullish**: Price makes lower low, RSI makes higher low → Bullish reversal
2. **Regular Bearish**: Price makes higher high, RSI makes lower high → Bearish reversal
3. **Hidden Bullish**: Price makes higher low, RSI makes lower low → Trend continuation (bullish)
4. **Hidden Bearish**: Price makes lower high, RSI makes higher high → Trend continuation (bearish)

**GARCH Enhancement:**

We use volatility context to filter divergence signals:

- **Confirmation**: Divergence + decreasing volatility → Stronger signal
- **Rejection**: Divergence + spiking volatility → Unreliable (wait)
- **Position Sizing**: Scale by inverse volatility

This improves the classic divergence strategy by avoiding false signals during volatile periods.

In [ ]:
# Detect divergences
from scipy.signal import find_peaks

# Find local extrema in price and RSI
lookback = 20  # Window for finding peaks

# Price extrema
price_highs_idx, _ = find_peaks(df_trading['Price'].values, distance=lookback)
price_lows_idx, _ = find_peaks(-df_trading['Price'].values, distance=lookback)

# RSI extrema
rsi_highs_idx, _ = find_peaks(df_trading['RSI'].values, distance=lookback)
rsi_lows_idx, _ = find_peaks(-df_trading['RSI'].values, distance=lookback)

def detect_regular_bullish_divergence(price, rsi, price_lows, rsi_lows):
    """Detect regular bullish divergence: price LL, RSI HL"""
    divergences = []
    for i in range(len(price_lows) - 1):
        idx1 = price_lows[i]
        idx2 = price_lows[i + 1]
        
        # Find corresponding RSI lows
        rsi_low1 = None
        rsi_low2 = None
        for j in rsi_lows:
            if abs(j - idx1) < 5:  # Allow small offset
                rsi_low1 = j
            if abs(j - idx2) < 5:
                rsi_low2 = j
        
        if rsi_low1 is not None and rsi_low2 is not None:
            # Check for divergence
            price_lower = price[idx2] < price[idx1]
            rsi_higher = rsi[rsi_low2] > rsi[rsi_low1]
            
            if price_lower and rsi_higher:
                divergences.append(idx2)
    
    return divergences

def detect_regular_bearish_divergence(price, rsi, price_highs, rsi_highs):
    """Detect regular bearish divergence: price HH, RSI LH"""
    divergences = []
    for i in range(len(price_highs) - 1):
        idx1 = price_highs[i]
        idx2 = price_highs[i + 1]
        
        # Find corresponding RSI highs
        rsi_high1 = None
        rsi_high2 = None
        for j in rsi_highs:
            if abs(j - idx1) < 5:
                rsi_high1 = j
            if abs(j - idx2) < 5:
                rsi_high2 = j
        
        if rsi_high1 is not None and rsi_high2 is not None:
            # Check for divergence
            price_higher = price[idx2] > price[idx1]
            rsi_lower = rsi[rsi_high2] < rsi[rsi_high1]
            
            if price_higher and rsi_lower:
                divergences.append(idx2)
    
    return divergences

# Detect divergences
bullish_div = detect_regular_bullish_divergence(
    df_trading['Price'].values, 
    df_trading['RSI'].values, 
    price_lows_idx, 
    rsi_lows_idx
)

bearish_div = detect_regular_bearish_divergence(
    df_trading['Price'].values, 
    df_trading['RSI'].values, 
    price_highs_idx, 
    rsi_highs_idx
)

# Filter divergences by volatility
vol_values = df_trading['Volatility'].values
vol_increasing_threshold = 0.02  # 2% vol threshold

bullish_div_filtered = [idx for idx in bullish_div 
                        if vol_values[idx] < vol_increasing_threshold]
bearish_div_filtered = [idx for idx in bearish_div 
                        if vol_values[idx] < vol_increasing_threshold]

print("="*70)
print("DIVERGENCE DETECTION + GARCH FILTER")
print("="*70)
print(f"\nTotal Regular Bullish Divergences: {len(bullish_div)}")
print(f"  Filtered (low vol): {len(bullish_div_filtered)}")
print(f"  Rejected (high vol): {len(bullish_div) - len(bullish_div_filtered)}")

print(f"\nTotal Regular Bearish Divergences: {len(bearish_div)}")
print(f"  Filtered (low vol): {len(bearish_div_filtered)}")
print(f"  Rejected (high vol): {len(bearish_div) - len(bearish_div_filtered)}")

# Show examples
if len(bullish_div_filtered) > 0:
    print(f"\nLast Bullish Divergence (filtered):")
    idx = bullish_div_filtered[-1]
    print(f"  Date: {df_trading['Date'].iloc[idx].strftime('%Y-%m-%d')}")
    print(f"  Price: ${df_trading['Price'].iloc[idx]:.2f}")
    print(f"  RSI: {df_trading['RSI'].iloc[idx]:.1f}")
    print(f"  Volatility: {df_trading['Volatility'].iloc[idx]*100:.2f}%")

if len(bearish_div_filtered) > 0:
    print(f"\nLast Bearish Divergence (filtered):")
    idx = bearish_div_filtered[-1]
    print(f"  Date: {df_trading['Date'].iloc[idx].strftime('%Y-%m-%d')}")
    print(f"  Price: ${df_trading['Price'].iloc[idx]:.2f}")
    print(f"  RSI: {df_trading['RSI'].iloc[idx]:.1f}")
    print(f"  Volatility: {df_trading['Volatility'].iloc[idx]*100:.2f}%")

# Visualize divergences
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Price with divergence markers
ax1 = axes[0]
ax1.plot(df_trading['Date'], df_trading['Price'], 'k-', linewidth=1.5, label='SPY Price')

# Mark all divergences (faded)
for idx in bullish_div:
    ax1.scatter(df_trading['Date'].iloc[idx], df_trading['Price'].iloc[idx], 
                color='green', marker='^', s=100, alpha=0.3, edgecolors='none')
for idx in bearish_div:
    ax1.scatter(df_trading['Date'].iloc[idx], df_trading['Price'].iloc[idx], 
                color='red', marker='v', s=100, alpha=0.3, edgecolors='none')

# Mark filtered divergences (bright)
for idx in bullish_div_filtered:
    ax1.scatter(df_trading['Date'].iloc[idx], df_trading['Price'].iloc[idx], 
                color='green', marker='^', s=200, alpha=1.0, 
                edgecolors='darkgreen', linewidths=2, label='Bullish Div (filtered)', zorder=5)
for idx in bearish_div_filtered:
    ax1.scatter(df_trading['Date'].iloc[idx], df_trading['Price'].iloc[idx], 
                color='red', marker='v', s=200, alpha=1.0, 
                edgecolors='darkred', linewidths=2, label='Bearish Div (filtered)', zorder=5)

# Remove duplicate labels
handles, labels = ax1.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax1.legend(by_label.values(), by_label.keys())

ax1.set_ylabel('Price ($)')
ax1.set_title('Divergence Detection with GARCH Volatility Filter', 
              fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

# Plot 2: RSI
ax2 = axes[1]
ax2.plot(df_trading['Date'], df_trading['RSI'], 'purple', linewidth=1.5)
ax2.axhline(y=70, color='red', linestyle='--', alpha=0.5)
ax2.axhline(y=30, color='green', linestyle='--', alpha=0.5)
ax2.set_ylabel('RSI')
ax2.set_title('RSI with Extrema', fontsize=12, fontweight='bold')
ax2.grid(alpha=0.3)
ax2.set_ylim(0, 100)

# Plot 3: Volatility
ax3 = axes[2]
ax3.plot(df_trading['Date'], df_trading['Volatility'] * 100, 'orange', linewidth=1.5)
ax3.axhline(y=vol_increasing_threshold*100, color='red', linestyle='--', linewidth=2,
            label=f'Vol Threshold ({vol_increasing_threshold*100:.0f}%)')

# Mark divergence periods
for idx in bullish_div_filtered:
    ax3.axvline(x=df_trading['Date'].iloc[idx], color='green', alpha=0.3, linestyle='-')
for idx in bearish_div_filtered:
    ax3.axvline(x=df_trading['Date'].iloc[idx], color='red', alpha=0.3, linestyle='-')

ax3.set_ylabel('Volatility (%)')
ax3.set_xlabel('Date')
ax3.set_title('GARCH Volatility (Filtering Criterion)', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSTRATEGY IMPROVEMENT:")
print("Without GARCH filter: All divergences taken (including during vol spikes)")
print("With GARCH filter: Only divergences in stable volatility regimes")
print("Result: Higher win rate, fewer false signals, better risk management")

---

## Part 12: Complete Mini-Project - Integrated Trading System

We now build a **complete end-to-end trading system** that integrates everything:

1. **GARCH(1,1)** for volatility forecasting
2. **RSI** for mean reversion signals
3. **Divergence detection** for reversal confirmation
4. **Dynamic position sizing** based on forecasted volatility
5. **Risk management** with volatility-adjusted stops

**System Architecture:**

```
Data → GARCH Model → Volatility Forecast
                           ↓
RSI Calculation → Signal Generation ← Divergence Detection
                           ↓
        Position Sizing (Vol-Adjusted)
                           ↓
        Trade Execution with Stops
                           ↓
        Performance Tracking & Analytics
```

**Entry Logic:**
- Long: (RSI < 30 OR Bullish Divergence) AND Vol < 2.5%
- Short: (RSI > 70 OR Bearish Divergence) AND Vol < 2.5%

**Exit Logic:**
- Stop-loss: 2 × forecasted σ
- Profit target: 3 × forecasted σ (risk-reward = 1.5)
- Neutral exit: RSI crosses 50

**Position Sizing:**
- Risk: 2% of portfolio per trade
- Size: Risk Capital / (2 × σ × Price)

In [ ]:
# Complete integrated trading system
class IntegratedTradingSystem:
    def __init__(self, prices, returns, vol_forecasts, rsi, divergences, 
                 portfolio_value=100000, risk_per_trade=0.02):
        self.prices = prices
        self.returns = returns
        self.vol_forecasts = vol_forecasts
        self.rsi = rsi
        self.bullish_div = set(divergences['bullish'])
        self.bearish_div = set(divergences['bearish'])
        self.portfolio_value = portfolio_value
        self.risk_per_trade = risk_per_trade
        
        self.position = 0  # 0=neutral, 1=long, -1=short
        self.entry_price = 0
        self.stop_loss = 0
        self.profit_target = 0
        self.trades = []
        self.equity_curve = [portfolio_value]
        
    def calculate_position_size(self, price, vol):
        """Calculate shares based on volatility"""
        risk_capital = self.portfolio_value * self.risk_per_trade
        dollar_risk_per_share = 2 * vol * price
        shares = int(risk_capital / dollar_risk_per_share)
        return max(shares, 1)  # At least 1 share
    
    def generate_signal(self, i):
        """Generate trading signal"""
        vol_threshold = 0.025  # 2.5%
        
        # Don't trade if volatility too high
        if self.vol_forecasts[i] > vol_threshold:
            return 0
        
        # Long conditions
        if self.rsi[i] < 30 or i in self.bullish_div:
            return 1
        
        # Short conditions
        if self.rsi[i] > 70 or i in self.bearish_div:
            return -1
        
        return 0
    
    def should_exit(self, i):
        """Check exit conditions"""
        current_price = self.prices[i]
        
        # Stop loss hit
        if self.position == 1 and current_price <= self.stop_loss:
            return True, "Stop Loss"
        if self.position == -1 and current_price >= self.stop_loss:
            return True, "Stop Loss"
        
        # Profit target hit
        if self.position == 1 and current_price >= self.profit_target:
            return True, "Profit Target"
        if self.position == -1 and current_price <= self.profit_target:
            return True, "Profit Target"
        
        # Neutral RSI
        if 45 <= self.rsi[i] <= 55:
            return True, "RSI Neutral"
        
        return False, None
    
    def run_backtest(self):
        """Execute backtest"""
        for i in range(50, len(self.prices)):  # Start after indicators stabilize
            current_price = self.prices[i]
            
            # Check exit if in position
            if self.position != 0:
                should_exit, exit_reason = self.should_exit(i)
                if should_exit:
                    # Close position
                    pnl = (current_price - self.entry_price) * self.position * self.shares
                    pnl_pct = (current_price / self.entry_price - 1) * self.position * 100
                    
                    self.portfolio_value += pnl
                    self.equity_curve.append(self.portfolio_value)
                    
                    self.trades.append({
                        'Entry_Date': self.entry_date,
                        'Exit_Date': i,
                        'Direction': 'Long' if self.position == 1 else 'Short',
                        'Entry_Price': self.entry_price,
                        'Exit_Price': current_price,
                        'Shares': self.shares,
                        'PnL': pnl,
                        'PnL_Pct': pnl_pct,
                        'Exit_Reason': exit_reason,
                        'Volatility': self.vol_forecasts[self.entry_date]
                    })
                    
                    self.position = 0
                else:
                    self.equity_curve.append(self.portfolio_value)
                continue
            
            # Generate new signal if no position
            signal = self.generate_signal(i)
            
            if signal != 0:
                # Enter position
                self.position = signal
                self.entry_price = current_price
                self.entry_date = i
                self.shares = self.calculate_position_size(current_price, self.vol_forecasts[i])
                
                # Set stops and targets
                vol_dollars = self.vol_forecasts[i] * current_price
                if signal == 1:  # Long
                    self.stop_loss = current_price - 2 * vol_dollars
                    self.profit_target = current_price + 3 * vol_dollars
                else:  # Short
                    self.stop_loss = current_price + 2 * vol_dollars
                    self.profit_target = current_price - 3 * vol_dollars
            
            self.equity_curve.append(self.portfolio_value)
        
        return self.trades, self.equity_curve

# Run the integrated system
system = IntegratedTradingSystem(
    prices=df_trading['Price'].values,
    returns=df_trading['Next_Return'].values,
    vol_forecasts=df_trading['Volatility'].values,
    rsi=df_trading['RSI'].values,
    divergences={'bullish': bullish_div_filtered, 'bearish': bearish_div_filtered}
)

trades, equity_curve = system.run_backtest()

# Convert trades to DataFrame
df_trades = pd.DataFrame(trades)

# Calculate performance metrics
if len(df_trades) > 0:
    winning_trades = df_trades[df_trades['PnL'] > 0]
    losing_trades = df_trades[df_trades['PnL'] < 0]
    
    win_rate = len(winning_trades) / len(df_trades) * 100
    avg_win = winning_trades['PnL'].mean() if len(winning_trades) > 0 else 0
    avg_loss = losing_trades['PnL'].mean() if len(losing_trades) > 0 else 0
    profit_factor = abs(winning_trades['PnL'].sum() / losing_trades['PnL'].sum()) if len(losing_trades) > 0 else np.inf
    
    total_return = (equity_curve[-1] / equity_curve[0] - 1) * 100
    max_drawdown = np.min(np.array(equity_curve) / np.maximum.accumulate(equity_curve) - 1) * 100
    
    print("="*70)
    print("INTEGRATED TRADING SYSTEM BACKTEST RESULTS")
    print("="*70)
    print(f"\nInitial Capital: ${equity_curve[0]:,.0f}")
    print(f"Final Capital: ${equity_curve[-1]:,.0f}")
    print(f"Total Return: {total_return:+.2f}%")
    print(f"Max Drawdown: {max_drawdown:.2f}%")
    
    print(f"\nTRADE STATISTICS:")
    print(f"  Total Trades: {len(df_trades)}")
    print(f"  Winning Trades: {len(winning_trades)} ({win_rate:.1f}%)")
    print(f"  Losing Trades: {len(losing_trades)}")
    print(f"  Average Win: ${avg_win:,.2f}")
    print(f"  Average Loss: ${avg_loss:,.2f}")
    print(f"  Profit Factor: {profit_factor:.2f}")
    print(f"  Risk-Reward Ratio: {abs(avg_win/avg_loss):.2f}" if avg_loss != 0 else "  Risk-Reward Ratio: N/A")
    
    print(f"\nBREAKDOWN BY DIRECTION:")
    long_trades = df_trades[df_trades['Direction'] == 'Long']
    short_trades = df_trades[df_trades['Direction'] == 'Short']
    print(f"  Long Trades: {len(long_trades)} (Win Rate: {len(long_trades[long_trades['PnL']>0])/len(long_trades)*100:.1f}%)" if len(long_trades) > 0 else "  Long Trades: 0")
    print(f"  Short Trades: {len(short_trades)} (Win Rate: {len(short_trades[short_trades['PnL']>0])/len(short_trades)*100:.1f}%)" if len(short_trades) > 0 else "  Short Trades: 0")
    
    print(f"\nEXIT REASONS:")
    for reason, count in df_trades['Exit_Reason'].value_counts().items():
        print(f"  {reason}: {count} ({count/len(df_trades)*100:.1f}%)")
    
    # Show best and worst trades
    print(f"\nBEST TRADE:")
    best = df_trades.loc[df_trades['PnL'].idxmax()]
    print(f"  {best['Direction']} | Entry: ${best['Entry_Price']:.2f} | Exit: ${best['Exit_Price']:.2f} | PnL: ${best['PnL']:,.2f} ({best['PnL_Pct']:+.2f}%)")
    
    print(f"\nWORST TRADE:")
    worst = df_trades.loc[df_trades['PnL'].idxmin()]
    print(f"  {worst['Direction']} | Entry: ${worst['Entry_Price']:.2f} | Exit: ${worst['Exit_Price']:.2f} | PnL: ${worst['PnL']:,.2f} ({worst['PnL_Pct']:+.2f}%)")
    
    # Visualize results
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    # Plot 1: Equity curve
    ax1 = axes[0]
    dates_equity = df_trading['Date'].iloc[:len(equity_curve)]
    ax1.plot(dates_equity, equity_curve, 'b-', linewidth=2, label='Strategy Equity')
    ax1.axhline(y=equity_curve[0], color='gray', linestyle='--', alpha=0.5, label='Starting Capital')
    ax1.set_ylabel('Portfolio Value ($)')
    ax1.set_title('Integrated Trading System: Equity Curve', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
    
    # Plot 2: Price with trades
    ax2 = axes[1]
    ax2.plot(df_trading['Date'], df_trading['Price'], 'k-', linewidth=1, alpha=0.5, label='SPY Price')
    
    for _, trade in df_trades.iterrows():
        entry_date = df_trading['Date'].iloc[trade['Entry_Date']]
        exit_date = df_trading['Date'].iloc[trade['Exit_Date']]
        entry_price = trade['Entry_Price']
        exit_price = trade['Exit_Price']
        
        color = 'green' if trade['PnL'] > 0 else 'red'
        marker = '^' if trade['Direction'] == 'Long' else 'v'
        
        ax2.scatter(entry_date, entry_price, color=color, marker=marker, s=100, alpha=0.7, edgecolors='black')
        ax2.scatter(exit_date, exit_price, color=color, marker='o', s=50, alpha=0.7)
        ax2.plot([entry_date, exit_date], [entry_price, exit_price], color=color, alpha=0.3, linestyle='--')
    
    ax2.set_ylabel('Price ($)')
    ax2.set_title('Trade Entries and Exits on Price Chart', fontsize=12, fontweight='bold')
    ax2.grid(alpha=0.3)
    
    # Plot 3: Trade PnL distribution
    ax3 = axes[2]
    ax3.bar(range(len(df_trades)), df_trades['PnL'], 
            color=['green' if x > 0 else 'red' for x in df_trades['PnL']], alpha=0.7)
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax3.set_xlabel('Trade Number')
    ax3.set_ylabel('PnL ($)')
    ax3.set_title('Individual Trade Profit/Loss', fontsize=12, fontweight='bold')
    ax3.grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("\nSYSTEM HIGHLIGHTS:")
    print("1. Combines multiple signals: RSI + Divergences + GARCH")
    print("2. Dynamic position sizing adapts to market volatility")
    print("3. Multi-level exits: Stops, Targets, and Neutral signals")
    print("4. Risk-managed approach: 2% risk per trade, vol-adjusted")
    print("5. Professional-grade system ready for live trading (with refinements)")
else:
    print("No trades generated. Adjust strategy parameters.")

---

## Part 13: Exercises and Challenges

Now it's your turn to practice and extend what you've learned.

### Exercise 1: Parameter Sensitivity Analysis
**Task**: Analyze how GARCH parameters affect volatility forecasts
- Estimate GARCH(1,1) on a different time period (e.g., 2015-2019)
- Compare parameters with our 2020-2024 estimation
- Question: How did the COVID-19 crisis change persistence (α + β)?

**Hint**: Higher persistence after crisis suggests longer-lasting volatility shocks.

---

### Exercise 2: Model Selection
**Task**: Compare different GARCH variants
- Estimate: ARCH(2), GARCH(1,1), GARCH(2,2), EGARCH(1,1)
- Use AIC/BIC for model selection
- Test forecasting accuracy on out-of-sample data

**Code Template**:
```python
from arch import arch_model

# GARCH(2,2)
model_22 = arch_model(returns, vol='GARCH', p=2, q=2)
result_22 = model_22.fit()
print(f"AIC: {result_22.aic}, BIC: {result_22.bic}")
```

**Question**: Does added complexity improve forecasts?

---

### Exercise 3: Volatility Regime Classification
**Task**: Use GARCH forecasts to classify market regimes
- Define regimes: Low (σ < 1%), Medium (1-2%), High (2-3%), Extreme (>3%)
- Calculate average returns in each regime
- Question: Are low-volatility regimes followed by high returns?

**Hint**: Use `pd.cut()` to bin volatility values.

---

### Exercise 4: Options Trading Strategy
**Task**: Build a volatility-based options strategy
- Compare GARCH forecasts with VIX (implied volatility)
- Signal: Long straddle when GARCH forecast > VIX (volatility underpriced)
- Calculate theoretical P&L assuming perfect hedging

**Extension**: Incorporate EGARCH asymmetry for put/call weighting

---

### Exercise 5: Multi-Asset Portfolio
**Task**: Apply GARCH to portfolio risk management
- Download 3-5 assets (stocks, bonds, commodities)
- Estimate GARCH(1,1) for each
- Build dynamic portfolio weights: $w_i \propto 1/\sigma_i^2$ (inverse volatility)
- Compare performance vs equal-weighted portfolio

**Metrics to report**:
- Sharpe ratio
- Max drawdown
- Turnover (rebalancing frequency)

---

### Challenge 1: Real-Time Trading System
**Advanced Task**: Build a production-ready system
1. Fetch live data (use `yfinance` with recent dates)
2. Update GARCH model daily (rolling window)
3. Generate signals and position sizes
4. Log all trades to a database/CSV
5. Send alerts (email/SMS) on trade execution

**Technologies**: Python + SQLite + APScheduler (for scheduling) + smtplib (for emails)

---

### Challenge 2: Deep Learning Enhancement
**Research Task**: Replace GARCH with Neural Network
- Architecture: LSTM for volatility forecasting
- Input: Lagged returns, rolling statistics
- Output: Next-day volatility
- Compare with GARCH(1,1) on MAE, RMSE

**Question**: Does deep learning outperform classical econometric models?

---

### Challenge 3: High-Frequency Data
**Advanced Analysis**: Apply GARCH to intraday data
- Download 1-minute or 5-minute SPY data
- Estimate GARCH on intraday returns
- Question: How do parameters differ from daily frequency?
- Use realized volatility (RV) as benchmark

**Key concept**: At higher frequencies, microstructure noise matters.

---

### Challenge 4: Multivariate GARCH
**Extension**: Model volatility spillovers across assets
- Estimate DCC-GARCH (Dynamic Conditional Correlation)
- Assets: SPY, TLT (bonds), GLD (gold)
- Analyze time-varying correlations during stress periods
- Build crisis-robust portfolio using correlation forecasts

**Library**: `arch` supports multivariate GARCH through `DCC` class

---

### Solutions and Discussion

Work through these exercises to deepen your understanding. Compare your results with peers or reference materials. Key learning goals:

1. **Sensitivity**: Understanding how parameters change across regimes
2. **Model Selection**: Balancing complexity and performance
3. **Application**: Translating theory to practical trading
4. **Extension**: Pushing beyond basics to frontier methods

Remember: The best way to learn quantitative finance is by coding and experimenting!

---

## Summary and Key Takeaways

Congratulations on completing this comprehensive GARCH volatility modeling tutorial.

### What We Covered

1. **Volatility Clustering**: Financial returns exhibit time-varying volatility with periods of high and low volatility that cluster together

2. **ARCH Models**: Capture volatility clustering by modeling conditional variance as a function of past squared returns

3. **GARCH Models**: Extend ARCH by including past conditional variances, creating persistence in volatility shocks

4. **EGARCH Models**: Incorporate asymmetry (leverage effect) where negative shocks increase volatility more than positive shocks

5. **Forecasting**: Generate multi-step ahead volatility forecasts that mean-revert to long-run levels

6. **Position Sizing**: Scale trading positions inversely with forecasted volatility to maintain consistent risk

7. **Technical Integration**: Combine GARCH with RSI and divergence analysis for robust trading signals

8. **Complete System**: Build an end-to-end trading system with dynamic risk management

### Critical Insights

**For Traders:**
- Use GARCH forecasts for position sizing, not just signal generation
- Volatility regimes are as important as price direction
- Risk management is dynamic, not static
- Asymmetry matters: Protect downside more than upside

**For Risk Managers:**
- GARCH provides better VaR estimates than historical methods
- Volatility persistence affects capital requirements
- Model validation requires out-of-sample testing
- Extreme events need separate tail risk models

**For Researchers:**
- Classical econometric models remain competitive with ML
- Domain knowledge (asymmetry, clustering) beats black-box approaches
- Parsimony wins: GARCH(1,1) often sufficient
- Always validate assumptions (normality, stationarity)

### Next Steps

1. **Practice**: Work through the exercises to solidify understanding
2. **Extend**: Try multivariate models (DCC-GARCH) for portfolio applications
3. **Combine**: Integrate with machine learning for hybrid systems
4. **Deploy**: Build a live trading system with proper infrastructure

### Recommended Resources

**Books:**
- Ruey S. Tsay - "Analysis of Financial Time Series"
- Carol Alexander - "Market Risk Analysis, Volume II"
- John C. Hull - "Options, Futures, and Other Derivatives"

**Papers:**
- Bollerslev (1986) - Original GARCH paper
- Nelson (1991) - EGARCH specification
- Engle (2002) - Dynamic Conditional Correlation

**Software:**
- Python `arch` library: https://arch.readthedocs.io/
- R `rugarch` package for advanced specifications
- MATLAB Econometrics Toolbox

### Final Thoughts

Volatility modeling is both an art and a science. The models provide structure, but practical application requires judgment, experience, and continuous learning. Markets evolve, and so must our models.

The integration of classical econometrics (GARCH) with modern techniques (ML, deep learning) represents the frontier of quantitative finance. Master the fundamentals here, then explore the cutting edge.

**Remember**: "In theory, there is no difference between theory and practice. In practice, there is." - Yogi Berra

Good luck in your quantitative trading journey!